In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/all_results_janus_single_node_1-14-17.csv')

In [3]:
df.head(5)

,num_procs,matrix_name,solver,preconditioner,status,time,iterations,final_residual
0,1,saylr3.mtx,FIXED_POINT,ILUT,unconverged,0.806348,10000.0,0.260768
1,1,saylr3.mtx,FIXED_POINT,RILUK,error,0.001645,NaN,NaN
2,1,saylr3.mtx,FIXED_POINT,RELAXATION,unconverged,0.325009,10000.0,0.063474
3,1,saylr3.mtx,FIXED_POINT,CHEBYSHEV,error,0.007176,NaN,NaN
4,1,saylr3.mtx,FIXED_POINT,NONE,error,0.011856,NaN,NaN


Changing the name of the columns to more simple names for ease-of-use.

In [4]:
df.columns= ['np', 'matrix', 'solver', 'prec', 'status', 'time', 'iters', 'resid']

In [5]:
df.head(5)

,np,matrix,solver,prec,status,time,iters,resid
0,1,saylr3.mtx,FIXED_POINT,ILUT,unconverged,0.806348,10000.0,0.260768
1,1,saylr3.mtx,FIXED_POINT,RILUK,error,0.001645,NaN,NaN
2,1,saylr3.mtx,FIXED_POINT,RELAXATION,unconverged,0.325009,10000.0,0.063474
3,1,saylr3.mtx,FIXED_POINT,CHEBYSHEV,error,0.007176,NaN,NaN
4,1,saylr3.mtx,FIXED_POINT,NONE,error,0.011856,NaN,NaN


How many total rows do we have?

In [6]:
len(df)

285275

Attempt at getting some basic info from the timings

In [7]:
df.describe()

/home/pate/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,np,time,iters,resid
count,285275.000000,285275.000000,163166.000000,1.245290e+05
mean,6.344695,14.156326,6306.703229,inf
std,3.752068,83.139410,4427.508800,NaN
min,1.000000,0.000297,1.000000,0.000000e+00
25%,2.000000,0.006834,NaN,NaN
50%,6.000000,0.234893,NaN,NaN
75%,10.000000,3.542865,NaN,NaN
max,12.000000,4288.670000,10000.000000,inf


We can now access the modified column names like an object

In [8]:
df.time

0            0.806348
1            0.001645
2            0.325009
3            0.007176
4            0.011856
5            0.198511
6            0.001454
7            1.143130
8            0.974769
9            1.649560
10           0.420352
11           0.417270
12           0.415508
13           0.417506
14           0.413472
15           0.001205
16           0.001093
17           0.000310
18           0.000678
19           0.000307
20           0.001208
21           0.001163
22           0.000352
23           0.000701
24           0.000316
25           0.154298
26           0.001435
27           1.031930
28           1.016320
29           2.234580
             ...     
285245    1594.720000
285246    1581.760000
285247     835.426000
285248     945.718000
285249     900.731000
285250    1690.600000
285251    1773.110000
285252     937.361000
285253     954.732000
285254     909.077000
285255    1235.320000
285256       2.823500
285257    1501.500000
285258    1534.060000
285259    